In [ ]:
import requests
import json
import datetime
import csv
import time

In [ ]:
page_id = "babygurunutrition/community"

# go to https://developers.facebook.com/tools/explorer/ to retreive a temp access token for testing purpose
access_token = "your access token"

In [ ]:
def getPage(page_id, access_token):
    base = "https://graph.facebook.com/v2.11"
    node = "/" + page_id
    params = "?access_token=%s" % access_token
    url = base + node + params
    
    print(url)
    
    r = requests.get(url)
    return r

In [ ]:
getPage(page_id, access_token)

In [ ]:
def processFacebookPageFeedStatus(status):
    
    if 'created_time' in status.keys():
        created_time = status['created_time'] 
    else:
        created_time = ''
    
    if 'description' in status.keys():
        description = status['description']
    else:
        description = ''
        
    if 'full_picture' in status.keys():    
        full_picture = status['full_picture']
    else:
        full_picture = ''
        
    if 'id' in status.keys():        
        id = status['id']
    else:
        id = ''
        
    if 'link' in status.keys():   
        link = status['link']
    else:
        link = ''
        
    if 'message' in status.keys():
        message = status['message']
    else:
        message = ''
        
    if 'name' in status.keys():    
        name = status['name']
    else:
        name = ''
        
    if 'type' in status.keys():
        type = status['type']
    else:
        type = ''
    
    # return a tuple of all processed data
    return (created_time, description, full_picture, id, link,
           message, name, type)

In [ ]:
def queryFB (page_id, access_token):
    """ retrieve the statuses of one page for a specified time range"""
    # format since = "YYYY-MM-DD"T"HH:MM:SS+XXXX" (XXXX = time zone difference)
    # format until = "YYYY-MM-DD"T"HH:MM:SS"
    # format access_token = [app_id] + "|" + [app_secret]       

    # build URL
    base = "https://graph.facebook.com/v2.11"
    node = "/" + page_id + "/tagged"
    params = "?fields=link,created_time,type,name,id,description,full_picture,message,status_type&limit=%s&access_token=%s"\
    % (100, access_token)

    url = base + node + params
    print(url)
        
    r = requests.get(url)
    
    data = json.loads(r.text)
    return (data,url)

In [ ]:
queryFB('babygurunutrition', access_token)

In [ ]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_statuses.csv' %page_id, 'w', newline='') as f:
        w = csv.writer(f)
        w. writerow(['created_time','description','full_picture','id','link','message','name', 'type'])
        
        has_next_page = True
        num_processed = 0
        
        statuses,url = queryFB(page_id, access_token)
        
        while has_next_page:
            for status in statuses['data']:
                w.writerow(processFacebookPageFeedStatus(status))
                
                num_processed += 1
                
            if 'paging' in statuses.keys() and 'next' in statuses['paging'].keys():
                statuses = requests.get(statuses['paging']['next']).json()
            else:
                has_next_page = False
        
        print('%s statuses has been collected for %s' %(num_processed, page_id))

In [ ]:
scrapeFacebookPageFeedStatus(page_id, access_token)